In [9]:
import pandas as pd
import polars as pl


In [10]:
cn = pd.read_csv("cn_data.csv")
pn = pd.read_csv("pn_data.csv")
prod = pl.read_csv(r"C:\WorkSpace\Python\DA_basics\Data\products.csv", infer_schema_length=10000, truncate_ragged_lines=True)


In [11]:
cn = cn.sort_values(["key", "vendor_cn_date"]).reset_index(drop=True)
pn = pn.sort_values(["key", "date"]).reset_index(drop=True)

In [12]:
product_mapping = {row["ws_code"]: row["product_name"] for row in prod.to_dicts()}

In [13]:
data = []
i, j = 0, 0
for key in sorted(set(cn["key"].unique()).union(set(pn["key"].unique()))):
    vendor_id, ws_code, batch_num = key.split("_")
    product_name = product_mapping.get(int(ws_code), "")
    
    while True:
        if i < len(pn["key"]) and j < len(cn["key"]) and pn["key"][i] == key and cn["key"][j] == key:
            k = pn["quantity"][i] - cn["billed_quantity"][j]
            if k > 0:
                data.append({
                    "vendor_id": vendor_id,
                    "ws_code": ws_code,
                    "batch_number": batch_num,
                    "product_name": product_name,
                    "cn_number": cn["debit_note_number"][j],
                    "cn_date": cn["vendor_cn_date"][j],
                    "cn_ref_date": "",
                    "cn_qty": cn["billed_quantity"][j],
                    "pn_number": pn["pick_note_number"][i],
                    "pn_date": pn["date"][i],
                    "pn_qty": pn["quantity"][i],
                    "finalqty": k,
                    "Note": "CN pending",
                    "key": key
                })
                pn.loc[i, "quantity"] = k
                j += 1
            elif k < 0:
                data.append({
                    "vendor_id": vendor_id,
                    "ws_code": ws_code,
                    "batch_number": batch_num,
                    "product_name": product_name,
                    "cn_number": cn["debit_note_number"][j],
                    "cn_date": cn["vendor_cn_date"][j],
                    "cn_ref_date": "",
                    "cn_qty": cn["billed_quantity"][j],
                    "pn_number": pn["pick_note_number"][i],
                    "pn_date": pn["date"][i],
                    "pn_qty": pn["quantity"][i],
                    "finalqty": -k,
                    "Note": "Excess CN received",
                    "key": key
                })
                cn.loc[j, "billed_quantity"] = -k
                i += 1
            else:
                data.append({
                    "vendor_id": vendor_id,
                    "ws_code": ws_code,
                    "batch_number": batch_num,
                    "product_name": product_name,
                    "cn_number": cn["debit_note_number"][j],
                    "cn_date": cn["vendor_cn_date"][j],
                    "cn_ref_date": "",
                    "cn_qty": cn["billed_quantity"][j],
                    "pn_number": pn["pick_note_number"][i],
                    "pn_date": pn["date"][i],
                    "pn_qty": pn["quantity"][i],
                    "finalqty": k,
                    "Note": "Exact Match",
                    "key": key
                })
                i += 1
                j += 1
        elif i < len(pn["key"]) and pn["key"][i] == key:
            data.append({
                "vendor_id": vendor_id,
                "ws_code": ws_code,
                "batch_number": batch_num,
                "product_name": product_name,
                "cn_number": "",
                "cn_date": "",
                "cn_ref_date": "",
                "cn_qty": "",
                "pn_number": pn["pick_note_number"][i],
                "pn_date": pn["date"][i],
                "pn_qty": pn["quantity"][i],
                "finalqty": pn["quantity"][i],
                "Note": "CN pending",
                "key": key
            })
            i += 1
        elif j < len(cn["key"]) and cn["key"][j] == key:
            data.append({
                "vendor_id": vendor_id,
                "ws_code": ws_code,
                "batch_number": batch_num,
                "product_name": product_name,
                "cn_number": cn["debit_note_number"][j],
                "cn_date": cn["vendor_cn_date"][j],
                "cn_ref_date": "",
                "cn_qty": cn["billed_quantity"][j],
                "pn_number": "",
                "pn_date": "",
                "pn_qty": "",
                "finalqty": cn["billed_quantity"][j],
                "Note": "Excess CN received",
                "key": key
            })
            j += 1
        else:
            break


In [14]:
temp_df = pd.DataFrame(data)

In [15]:
temp_df.to_csv("output.csv")